# 02 - Data from the Web

## Deadline
Wednesday October 25, 2017 at 11:59PM

## Important Notes
* Make sure you push on GitHub your Notebook with all the cells already evaluated (i.e., you don't want your colleagues to generate unnecessary Web traffic during the peer review)
* Don't forget to add a textual description of your thought process, the assumptions you made, and the solution you plan to implement!
* Please write all your comments in English, and use meaningful variable names in your code.

## Background
In this homework we will extract interesting information from www.topuniversities.com and www.timeshighereducation.com, two platforms that maintain a global ranking of worldwide universities. This ranking is not offered as a downloadable dataset, so you will have to find a way to scrape the information we need!
You are not allowed to download manually the entire ranking -- rather you have to understand how the server loads it in your browser. For this task, Postman with the Interceptor extension can help you greatly. We recommend that you watch this [brief tutorial](https://www.youtube.com/watch?v=jBjXVrS8nXs&list=PLM-7VG-sgbtD8qBnGeQM5nvlpqB_ktaLZ&autoplay=1) to understand quickly how to use it.

## Assignment
1. Obtain the 200 top-ranking universities in www.topuniversities.com ([ranking 2018](https://www.topuniversities.com/university-rankings/world-university-rankings/2018)). In particular, extract the following fields for each university: name, rank, country and region, number of faculty members (international and total) and number of students (international and total). Some information is not available in the main list and you have to find them in the [details page](https://www.topuniversities.com/universities/ecole-polytechnique-fédérale-de-lausanne-epfl).
Store the resulting dataset in a pandas DataFrame and answer the following questions:
- Which are the best universities in term of: (a) ratio between faculty members and students, (b) ratio of international students?
- Answer the previous question aggregating the data by (c) country and (d) region.

Plot your data using bar charts and describe briefly what you observed.

2. Obtain the 200 top-ranking universities in www.timeshighereducation.com ([ranking 2018](http://timeshighereducation.com/world-university-rankings/2018/world-ranking)). Repeat the analysis of the previous point and discuss briefly what you observed.

3. Merge the two DataFrames created in questions 1 and 2 using university names. Match universities' names as well as you can, and explain your strategy. Keep track of the original position in both rankings.

4. Find useful insights in the data by performing an exploratory analysis. Can you find a strong correlation between any pair of variables in the dataset you just created? Example: when a university is strong in its international dimension, can you observe a consistency both for students and faculty members?

5. Can you find the best university taking in consideration both rankings? Explain your approach.

Hints:
- Keep your Notebook clean and don't print the verbose output of the requests if this does not add useful information for the reader.
- In case of tie, use the order defined in the webpage.


# Solution
First let's try to do the request on the https://www.topuniversities.com/university-rankings/world-university-rankings/2018 url just to see what we get.

In [1]:
import requests
from bs4 import BeautifulSoup
r = requests.get('https://www.topuniversities.com/university-rankings/world-university-rankings/2018')
soup = BeautifulSoup(r.text, 'html.parser')
soup

<!DOCTYPE html>

<html dir="ltr" version="XHTML+RDFa 1.0" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml" xmlns:article="http://ogp.me/ns/article#" xmlns:book="http://ogp.me/ns/book#" xmlns:content="http://purl.org/rss/1.0/modules/content/" xmlns:dc="http://purl.org/dc/terms/" xmlns:foaf="http://xmlns.com/foaf/0.1/" xmlns:og="http://ogp.me/ns#" xmlns:product="http://ogp.me/ns/product#" xmlns:profile="http://ogp.me/ns/profile#" xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#" xmlns:schema="http://schema.org/" xmlns:sioc="http://rdfs.org/sioc/ns#" xmlns:sioct="http://rdfs.org/sioc/types#" xmlns:skos="http://www.w3.org/2004/02/skos/core#" xmlns:video="http://ogp.me/ns/video#" xmlns:xsd="http://www.w3.org/2001/XMLSchema#">
<head profile="http://www.w3.org/1999/xhtml/vocab">
<meta content="unsafe-url" name="referrer"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><script type="text/javascript">(window.NREUM||(NREUM={})).loader_config={xpid:"UwUCVVVTGwIAV1VXBQkP"}

If we do Ctrl+F and try to look for "Imperial College London" for example, we don't get anything in the Beautifoul soup HTML even if the string is clearly on the webpage. This is because the data is not loaded at the beginning when the HTML request is answered. It is loaded later and the ranking data is in a separate file. Using Google Chrome Interceptor we figured out the file is at https://www.topuniversities.com/sites/default/files/qs-rankings-data/357051_indicators.txt. So this is the webpage we will scrape instead of the ranking webpage:

In [2]:
from html import parser
import os

base_url = 'https://www.topuniversities.com/'

# create a subclass and override the handler methods
class QSHTMLParser(parser.HTMLParser):
    def handle_starttag(self, tag, attrs):
        if not hasattr(self, 'array_hrefs'):
            self.array_hrefs = []
        if tag == 'a':
            self.last_tag_url = os.path.join(base_url, attrs[0][1][1:])
            
    def handle_data(self, data):
        self.array_hrefs.append({"url": self.last_tag_url, "name": data})

In [3]:
import json

request_ranking_text_file = requests.get('https://www.topuniversities.com/sites/default/files/qs-rankings-data/357051_indicators.txt')
json_ranking = json.loads(request_ranking_text_file.text)
ranking_data = json_ranking['data'][:200]
parser = QSHTMLParser()
for university in ranking_data:
    parser.feed(university['uni'])
    
basic_data = parser.array_hrefs
for i, university in enumerate(ranking_data):
    curr_uni = basic_data[i]
    curr_uni['region'] = university['region']
    curr_uni['location'] = university['location']

Now we have the URLs of the fifty first universities:

In [4]:
basic_data

[{'location': 'United States',
  'name': 'Massachusetts Institute of Technology (MIT) ',
  'region': 'North America',
  'url': 'https://www.topuniversities.com/universities/massachusetts-institute-technology-mit'},
 {'location': 'United States',
  'name': 'Stanford University',
  'region': 'North America',
  'url': 'https://www.topuniversities.com/universities/stanford-university'},
 {'location': 'United States',
  'name': 'Harvard University',
  'region': 'North America',
  'url': 'https://www.topuniversities.com/universities/harvard-university'},
 {'location': 'United States',
  'name': 'California Institute of Technology (Caltech)',
  'region': 'North America',
  'url': 'https://www.topuniversities.com/universities/california-institute-technology-caltech'},
 {'location': 'United Kingdom',
  'name': 'University of Cambridge',
  'region': 'Europe',
  'url': 'https://www.topuniversities.com/universities/university-cambridge'},
 {'location': 'United Kingdom',
  'name': 'University of Ox

Here we have the URLs for the details pages for the 50 first universities. Let's take a closer look et the EPFL detail page:

In [5]:
import re

url_epfl = basic_data[11]['url']
request_epfl = requests.get(url_epfl)
soup_epfl = BeautifulSoup(request_epfl.text, 'html.parser')
div_details = soup_epfl.find_all('div', {"class": "view-academic-data-profile"})[0]
div_details.find_all('div', {"class": "number"})
test_det = div_details.find_all('div', {"class": "number"})
student_values = ["staff_total", "staff_international", "students_total", "students_international"]
number_students = {}
for i in range(0, 4):
    number = int(re.sub('[,]', '', test_det[i].get_text()))
    number_students[student_values[i]] = number
number_students

{'staff_international': 1300,
 'staff_total': 1695,
 'students_international': 5896,
 'students_total': 10343}

That's it, the code is maybe a little messy but with some browser inspection we can figure out where the things we need on the EPFL webpage are.
Now let us generalize this to all the 50 first universities:

In [6]:
import re

number_students_per_fac = {}
for i, data in enumerate(basic_data):
    url = data['url']
    print("Processing ", i, ": ", url)
    request_uni = requests.get(url)
    soup_uni = BeautifulSoup(request_uni.text, 'html.parser')
    div_details = soup_uni.find_all('div', {"class": "view-academic-data-profile"})[0]
    test_det = div_details.find_all('div', {"class": "number"})
    student_values = ["staff_total", "staff_international", "students_total", "students_international"]
    number_students = {}
    if(len(test_det) == 4):
        for j in range(0, 4):
            number = int(re.sub('[,]', '', test_det[j].get_text()))
            basic_data[i][student_values[j]] = number
    number_students_per_fac[url] = number_students

Processing  0 :  https://www.topuniversities.com/universities/massachusetts-institute-technology-mit
Processing  1 :  https://www.topuniversities.com/universities/stanford-university
Processing  2 :  https://www.topuniversities.com/universities/harvard-university
Processing  3 :  https://www.topuniversities.com/universities/california-institute-technology-caltech
Processing  4 :  https://www.topuniversities.com/universities/university-cambridge
Processing  5 :  https://www.topuniversities.com/universities/university-oxford
Processing  6 :  https://www.topuniversities.com/universities/ucl-university-college-london
Processing  7 :  https://www.topuniversities.com/universities/imperial-college-london
Processing  8 :  https://www.topuniversities.com/universities/university-chicago
Processing  9 :  https://www.topuniversities.com/universities/eth-zurich-swiss-federal-institute-technology
Processing  10 :  https://www.topuniversities.com/universities/nanyang-technological-university-singapor

In [7]:
basic_data

[{'location': 'United States',
  'name': 'Massachusetts Institute of Technology (MIT) ',
  'region': 'North America',
  'staff_international': 1679,
  'staff_total': 2982,
  'students_international': 3717,
  'students_total': 11067,
  'url': 'https://www.topuniversities.com/universities/massachusetts-institute-technology-mit'},
 {'location': 'United States',
  'name': 'Stanford University',
  'region': 'North America',
  'staff_international': 2042,
  'staff_total': 4285,
  'students_international': 3611,
  'students_total': 15878,
  'url': 'https://www.topuniversities.com/universities/stanford-university'},
 {'location': 'United States',
  'name': 'Harvard University',
  'region': 'North America',
  'staff_international': 1311,
  'staff_total': 4350,
  'students_international': 5266,
  'students_total': 22429,
  'url': 'https://www.topuniversities.com/universities/harvard-university'},
 {'location': 'United States',
  'name': 'California Institute of Technology (Caltech)',
  'region':

That's it, we have all the data for the number of students. We can convince ourselves that this is right by taking a university at random and visiting the corresponding webpage (if we take more of them the probability that our results our wrong decrease exponentially).

Now we simply construct our dataframe:

In [15]:
import pandas as pd
qs_dataframe = pd.DataFrame.from_dict(basic_data)
qs_dataframe

,location,name,region,staff_international,staff_total,students_international,students_total,url
0,United States,Massachusetts Institute of Technology (MIT),North America,1679.0,2982.0,3717.0,11067.0,https://www.topuniversities.com/universities/m...
1,United States,Stanford University,North America,2042.0,4285.0,3611.0,15878.0,https://www.topuniversities.com/universities/s...
2,United States,Harvard University,North America,1311.0,4350.0,5266.0,22429.0,https://www.topuniversities.com/universities/h...
3,United States,California Institute of Technology (Caltech),North America,350.0,953.0,647.0,2255.0,https://www.topuniversities.com/universities/c...
4,United Kingdom,University of Cambridge,Europe,2278.0,5490.0,6699.0,18770.0,https://www.topuniversities.com/universities/u...
5,United Kingdom,University of Oxford,Europe,2964.0,6750.0,7353.0,19720.0,https://www.topuniversities.com/universities/u...
6,United Kingdom,UCL (University College London),Europe,2554.0,6345.0,14854.0,31080.0,https://www.topuniversities.com/universities/u...
7,United Kingdom,Imperial College London,Europe,2071.0,3930.0,8746.0,16090.0,https://www.topuniversities.com/universities/i...
8,United States,University of Chicago,North America,635.0,2449.0,3379.0,13557.0,https://www.topuniversities.com/universities/u...
9,Switzerland,ETH Zurich - Swiss Federal Institute of Techno...,Europe,1886.0,2477.0,7563.0,19815.0,https://www.topuniversities.com/universities/e...


That's it! There is just one thing we have to noctice here: there are two universities (NYU at index 51 and IISc at index 189) where we do not have the info about the number of students / staff members. And indeed, if we visit the webpages we see that the information is not there.
For NYU we don't have anything so we leave it like that (we will see how to deal with the NaN values depending on the questions later). For ISSc we can see 3 out of the 4 informations on the webpage so we will enter it manually:

In [9]:
qs_dataframe[qs_dataframe.isnull().any(axis=1)]

,location,name,region,staff_international,staff_total,students_international,students_total,url
51,United States,New York University (NYU),North America,NaN,NaN,NaN,NaN,https://www.topuniversities.com/universities/n...
189,India,Indian Institute of Science (IISc) Bangalore,Asia,NaN,NaN,NaN,NaN,https://www.topuniversities.com/universities/i...


For NYU we have no idea so we search on wikipedia and see there are 50027 students and 9768 members of the academic staff. If we search their website https://www.nyu.edu/students/communities-and-groups/international-students.html we see that "over 17,000 international students and scholars from over 140 different countries", from which we can somehow infer that there are approximately 2800 international members of the academic staff and 14200 international students (if the distribution is the same as the total students/staff then one person out of six in these 17000 is a member of the academic staff). Of course, this approximation could be wrong but it will not be a real problem since it is only one value in 200. As for IISc, we see that one value out of the four is NaN (number of international members of the academic staff). Since there are very few international students then we can infer that there are also even fewer members of the academic staff so let's say that number is 0:

In [16]:
qs_dataframe = qs_dataframe.set_value(51, 'staff_international', 2800).set_value(51, 'staff_total', 9768).set_value(51, 'students_international', 14200).set_value(51, 'students_total', 50027)
qs_dataframe = qs_dataframe.set_value(189, 'staff_international', 0).set_value(189, 'staff_total', 423).set_value(189, 'students_international', 47).set_value(189, 'students_total', 4071)

(Here we did it manually because there were only two universities which were problematic. Of course, if those cases were more frequent then we would n
Question 2: If we want the best universities in terms of ratio staff / students then we do:

In [55]:
qs_dataframe_ratio_staff = qs_dataframe.assign(ratio_staff = qs_dataframe['staff_total'] / qs_dataframe['students_total']).sort_values('ratio_staff', ascending=False)
qs_dataframe_ratio_staff.head()

,location,name,region,staff_international,staff_total,students_international,students_total,url,ratio_staff
3,United States,California Institute of Technology (Caltech),North America,350.0,953.0,647.0,2255.0,https://www.topuniversities.com/universities/c...,0.422616
15,United States,Yale University,North America,1708.0,4940.0,2469.0,12402.0,https://www.topuniversities.com/universities/y...,0.398323
5,United Kingdom,University of Oxford,Europe,2964.0,6750.0,7353.0,19720.0,https://www.topuniversities.com/universities/u...,0.342292
4,United Kingdom,University of Cambridge,Europe,2278.0,5490.0,6699.0,18770.0,https://www.topuniversities.com/universities/u...,0.292488
16,United States,Johns Hopkins University,North America,1061.0,4462.0,4105.0,16146.0,https://www.topuniversities.com/universities/j...,0.276353


Similarly, if we want the best universities in terms of ratio of international students then we do:

In [53]:
qs_dataframe_ratio_intl_students = qs_dataframe.assign(ratio_intl_students = qs_dataframe['students_international'] / qs_dataframe['students_total']).sort_values('ratio_intl_students', ascending=False)
qs_dataframe_ratio_intl_students.head()

,location,name,region,staff_international,staff_total,students_international,students_total,url,ratio_intl_students
34,United Kingdom,London School of Economics and Political Scien...,Europe,687.0,1088.0,6748.0,9760.0,https://www.topuniversities.com/universities/l...,0.691393
11,Switzerland,Ecole Polytechnique Fédérale de Lausanne (EPFL),Europe,1300.0,1695.0,5896.0,10343.0,https://www.topuniversities.com/universities/e...,0.570047
7,United Kingdom,Imperial College London,Europe,2071.0,3930.0,8746.0,16090.0,https://www.topuniversities.com/universities/i...,0.543567
199,Netherlands,Maastricht University,Europe,502.0,1277.0,8234.0,16385.0,https://www.topuniversities.com/universities/m...,0.502533
46,United States,Carnegie Mellon University,North America,425.0,1342.0,6385.0,13356.0,https://www.topuniversities.com/universities/c...,0.478062


Ratio staff by location:

In [59]:
qs_dataframe_ratio_staff.groupby('location').apply(pd.DataFrame.sort_values, 'ratio_staff', ascending=False)

location  \
location                           
Argentina     74       Argentina   
Australia     19       Australia   
              47       Australia   
              109      Australia   
              41       Australia   
              93       Australia   
              44       Australia   
              49       Australia   
              59       Australia   
              175      Australia   
Austria       153        Austria   
              181        Austria   
Belgium       184        Belgium   
              125        Belgium   
              70         Belgium   
              152        Belgium   
Brazil        120         Brazil   
              182         Brazil   
Canada        139         Canada   
              30          Canada   
              31          Canada   
              90          Canada   
              50          Canada   
              129         Canada   
              151         Canada   
Chile         138          Chile   
China         24           China   
              39           China   
              96           China   
              37           China   
...                          ...   
United States 21   United States   
              168  United States   
              54   United States   
              88   United States   
              52   United States   
              85   United States   
              12   United States   
              13   United States   
              80   United States   
              183  United States   
              177  United States   
              174  United States   
              38   United States   
              117  United States   
              128  United States   
              46   United States   
              26   United States   
              104  United States   
              162  United States   
              131  United States   
              92   United States   
              32   United States   
              148  United States   
              133  United States   
              68   United States   
              165  United States   
              66   United States   
              196  United States   
              69   United States   
              60   United States   

                                                              name  \
location                                                             
Argentina     74                 Universidad de Buenos Aires (UBA)   
Australia     19                The Australian National University   
              47                      The University of Queensland   
              109                       The University of Adelaide   
              41                       The University of Melbourne   
              93               The University of Western Australia   
              44   The University of New South Wales (UNSW Sydney)   
              49                          The University of Sydney   
              59                                 Monash University   
              175                  University of Technology Sydney   
Austria       153                             University of Vienna   
              181                  Vienna University of Technology   
Belgium       184                 Vrije Universiteit Brussel (VUB)   
              125                                 Ghent University   
              70                                         KU Leuven   
              152           Université catholique de Louvain (UCL)   
Brazil        120                        Universidade de São Paulo   
              182      Universidade Estadual de Campinas (Unicamp)   
Canada        139                              McMaster University   
              30                             University of Toronto   
              31                                 McGill University   
              90                             University of Alberta   
              50                    University of British Columbia   
     

Ratio staff by region:

In [60]:
qs_dataframe_ratio_staff.groupby('region').apply(pd.DataFrame.sort_values, 'ratio_staff', ascending=False)

location  \
region                             
Africa        190   South Africa   
Asia          71     South Korea   
              76           Japan   
              35           Japan   
              172   Saudi Arabia   
              10       Singapore   
              115          Japan   
              28           Japan   
              55           Japan   
              14       Singapore   
              114       Malaysia   
              127          Japan   
              24           China   
              25       Hong Kong   
              107    South Korea   
              121          Japan   
              48       Hong Kong   
              89     South Korea   
              36     South Korea   
              105    South Korea   
              39           China   
              144         Israel   
              154    South Korea   
              40     South Korea   
              96           China   
              62           Japan   
              37           China   
              45       Hong Kong   
              95       Hong Kong   
              191          Japan   
...                          ...   
North America 128  United States   
              46   United States   
              50          Canada   
              26   United States   
              104  United States   
              162  United States   
              131  United States   
              92   United States   
              129         Canada   
              32   United States   
              148  United States   
              133  United States   
              68   United States   
              165  United States   
              66   United States   
              196  United States   
              69   United States   
              151         Canada   
              60   United States   
Oceania       19       Australia   
              47       Australia   
              109      Australia   
              41       Australia   
              93       Australia   
              44       Australia   
              49       Australia   
              81     New Zealand   
              150    New Zealand   
              59       Australia   
              175      Australia   

                                                                name  \
region                                                                 
Africa        190                            University of Cape Town   
Asia          71   Pohang University of Science And Technology (P...   
              76                                   Tohoku University   
              35                                    Kyoto University   
              172       King Fahd University of Petroleum & Minerals   
              10   Nanyang Technological University, Singapore (NTU)   
              115                                  Nagoya University   
              28                             The University of Tokyo   
              55                       Tokyo Institute of Technology   
              14              National University of Singapore (NUS)   
              114                             Universiti Malaya (UM)   
              127                                  Kyushu University   
              24                                 Tsinghua University   
              25                         The University of Hong Kong   
              107                     Sungkyunkwan University (SKKU)   
              121                                Hokkaido University   
              48                        City University of Hong Kong   
              89                                    Korea University   
              36                           Seoul National University   
              105                                  Yonsei University   
              39                                    Fudan University   
              144                 The Hebrew University of Jerusalem   
              154          

Ratio international students by location:

In [58]:
qs_dataframe_ratio_intl_students.groupby('location').apply(pd.DataFrame.sort_values, 'ratio_intl_students', ascending=False)

location  \
location                           
Argentina     74       Argentina   
Australia     41       Australia   
              19       Australia   
              109      Australia   
              49       Australia   
              44       Australia   
              59       Australia   
              175      Australia   
              47       Australia   
              93       Australia   
Austria       153        Austria   
              181        Austria   
Belgium       184        Belgium   
              152        Belgium   
              70         Belgium   
              125        Belgium   
Brazil        182         Brazil   
              120         Brazil   
Canada        31          Canada   
              30          Canada   
              90          Canada   
              151         Canada   
              129         Canada   
              50          Canada   
              139         Canada   
Chile         138          Chile   
China         37           China   
              39           China   
              86           China   
              24           China   
...                          ...   
United States 68   United States   
              12   United States   
              38   United States   
              18   United States   
              99   United States   
              15   United States   
              52   United States   
              20   United States   
              27   United States   
              21   United States   
              32   United States   
              146  United States   
              26   United States   
              148  United States   
              85   United States   
              92   United States   
              168  United States   
              60   United States   
              117  United States   
              128  United States   
              54   United States   
              142  United States   
              162  United States   
              177  United States   
              66   United States   
              133  United States   
              183  United States   
              174  United States   
              196  United States   
              79   United States   

                                                              name  \
location                                                             
Argentina     74                 Universidad de Buenos Aires (UBA)   
Australia     41                       The University of Melbourne   
              19                The Australian National University   
              109                       The University of Adelaide   
              49                          The University of Sydney   
              44   The University of New South Wales (UNSW Sydney)   
              59                                 Monash University   
              175                  University of Technology Sydney   
              47                      The University of Queensland   
              93               The University of Western Australia   
Austria       153                             University of Vienna   
              181                  Vienna University of Technology   
Belgium       184                 Vrije Universiteit Brussel (VUB)   
              152           Université catholique de Louvain (UCL)   
              70                                         KU Leuven   
              125                                 Ghent University   
Brazil        182      Universidade Estadual de Campinas (Unicamp)   
              120                        Universidade de São Paulo   
Canada        31                                 McGill University   
              30                             University of Toronto   
              90                             University of Alberta   
              151                           University of Waterloo   
              129                          Université de Montréal    
     

Ratio international students by region:

In [57]:
qs_dataframe_ratio_intl_students.groupby('region').apply(pd.DataFrame.sort_values, 'ratio_intl_students', ascending=False)

location  \
region                             
Africa        190   South Africa   
Asia          25       Hong Kong   
              48       Hong Kong   
              10       Singapore   
              29       Hong Kong   
              14       Singapore   
              45       Hong Kong   
              95       Hong Kong   
              114       Malaysia   
              37           China   
              172   Saudi Arabia   
              39           China   
              154    South Korea   
              89     South Korea   
              105    South Korea   
              107    South Korea   
              86           China   
              24           China   
              55           Japan   
              75          Taiwan   
              115          Japan   
              28           Japan   
              127          Japan   
              62           Japan   
              113          China   
              76           Japan   
              35           Japan   
              160         Taiwan   
              36     South Korea   
              121          Japan   
...                          ...   
North America 26   United States   
              148  United States   
              85   United States   
              92   United States   
              168  United States   
              139         Canada   
              60   United States   
              117  United States   
              128  United States   
              54   United States   
              142  United States   
              162  United States   
              177  United States   
              66   United States   
              133  United States   
              183  United States   
              174  United States   
              196  United States   
              79   United States   
Oceania       41       Australia   
              19       Australia   
              109      Australia   
              49       Australia   
              44       Australia   
              59       Australia   
              175      Australia   
              81     New Zealand   
              47       Australia   
              93       Australia   
              150    New Zealand   

                                                                name  \
region                                                                 
Africa        190                            University of Cape Town   
Asia          25                         The University of Hong Kong   
              48                        City University of Hong Kong   
              10   Nanyang Technological University, Singapore (NTU)   
              29   The Hong Kong University of Science and Techno...   
              14              National University of Singapore (NUS)   
              45          The Chinese University of Hong Kong (CUHK)   
              95                The Hong Kong Polytechnic University   
              114                             Universiti Malaya (UM)   
              37                                   Peking University   
              172       King Fahd University of Petroleum & Minerals   
              39                                    Fudan University   
              154                                 Hanyang University   
              89                                    Korea University   
              105                                  Yonsei University   
              107                     Sungkyunkwan University (SKKU)   
              86                                 Zhejiang University   
              24                                 Tsinghua University   
              55                       Tokyo Institute of Technology   
              75                    National Taiwan University (NTU)   
              115                                  Nagoya University   
              28                             The University of Tokyo   
              127          

## Times Ranking
Now let's try to request the page for Times ranking:

In [19]:
r_times = requests.get('https://www.timeshighereducation.com/world-university-rankings/2018/world-ranking#!/page/0/length/25/sort_by/rank/sort_order/asc/cols/stats')
soup_times = BeautifulSoup(r_times.text, 'html.parser')

In [20]:
soup_times.text

'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nWorld University Rankings 2018 | Times Higher Education (THE)\n\n\n\n\n\n\n\n\n\n\n\n\n\n\njQuery.extend(Drupal.settings, {"basePath":"\\/","pathPrefix":"","ajaxPageState":{"theme":"the_responsive","theme_token":"J0qLQKB4HsSZstfJfUmnvlUWkNCMxB029iiuEOVBR6w","js":{"sites\\/default\\/files\\/minify\\/jquery.once.1.2.min.js":1,"sites\\/default\\/files\\/minify\\/the_data_rankings.1.10.12.min.js":1,"sites\\/default\\/files\\/minify\\/jquery.cookie.67fb34f6a866c40d0570.min.js":1,"sites\\/default\\/files\\/minify\\/notification.min.js":1,"sites\\/default\\/files\\/minify\\/scripts.min.js":1,"sites\\/default\\/files\\/minify\\/the-geography-extras.min.js":1,"sites\\/default\\/files\\/minify\\/most_viewed_commented.min.js":1,"sites\\/default\\/files\\/minify\\/paywall.min.js":1,"sites\\/default\\/files\\/minify\\/the_dfp.min.js":1,"sites\\/default\\/files\\/minify\\/caption-filter.min.js":1,"sites\\/default\\/files\\/minify

Again, if we do Ctrl + F and type "Oxford", we find only the Oxford in the title paragraph explaining the ranking but bot the one from the ranking itself. And we do the same thing we did for the QS ranking: we use interceptor and we get all the data on 'https://www.timeshighereducation.com/sites/default/files/the_data_rankings/world_university_rankings_2018_limit0_369a9045a203e176392b9fb8f8c1cb2a.json

In [21]:
import json
r_times_ranking = requests.get('https://www.timeshighereducation.com/sites/default/files/the_data_rankings/world_university_rankings_2018_limit0_369a9045a203e176392b9fb8f8c1cb2a.json')
soup_times_ranking = BeautifulSoup(r_times_ranking.text, 'html.parser')
data_times_ranking = json.loads(soup_times_ranking.text)
data_times_50_first = data_times_ranking['data'][:50]

Now we have all the data in our 'data_times_50_first' dictionary. Let's convert that to a pandas frame:

In [22]:
import pandas as pd
times_dataframe = pd.DataFrame.from_dict(data_times_50_first)

Let's take a look at our data frame:

In [23]:
times_dataframe.head()

,aliases,location,member_level,name,nid,rank,rank_order,record_type,scores_citations,scores_citations_rank,...,scores_research,scores_research_rank,scores_teaching,scores_teaching_rank,stats_female_male_ratio,stats_number_students,stats_pc_intl_students,stats_student_staff_ratio,subjects_offered,url
0,University of Oxford,United Kingdom,0,University of Oxford,468,1,10,master_account,99.1,15,...,99.5,1,86.7,5,46 : 54,"20,409",38%,11.2,"Archaeology,Art, Performing Arts & Design,Biol...",/world-university-rankings/university-oxford
1,University of Cambridge,United Kingdom,0,University of Cambridge,470,2,20,master_account,97.5,29,...,97.8,3,87.8,3,45 : 55,"18,389",35%,10.9,"Archaeology,Architecture,Art, Performing Arts ...",/world-university-rankings/university-cambridge
2,California Institute of Technology caltech,United States,0,California Institute of Technology,128779,=3,30,private,99.5,10,...,97.5,4,90.3,1,31 : 69,"2,209",27%,6.5,"Architecture,Biological Sciences,Business & Ma...",/world-university-rankings/california-institut...
3,Stanford University,United States,11,Stanford University,467,=3,40,private,99.9,4,...,96.7,5,89.1,2,42 : 58,"15,845",22%,7.5,"Archaeology,Architecture,Art, Performing Arts ...",/world-university-rankings/stanford-university
4,Massachusetts Institute of Technology,United States,0,Massachusetts Institute of Technology,471,5,50,private,100.0,1,...,91.9,9,87.3,4,37 : 63,"11,177",34%,8.7,"Architecture,Art, Performing Arts & Design,Bio...",/world-university-rankings/massachusetts-insti...


We see there are a lot of columns:

In [24]:
print("There are {} columns: {}".format(times_dataframe.shape[1], times_dataframe.columns))

There are 26 columns: Index(['aliases', 'location', 'member_level', 'name', 'nid', 'rank',
       'rank_order', 'record_type', 'scores_citations',
       'scores_citations_rank', 'scores_industry_income',
       'scores_industry_income_rank', 'scores_international_outlook',
       'scores_international_outlook_rank', 'scores_overall',
       'scores_overall_rank', 'scores_research', 'scores_research_rank',
       'scores_teaching', 'scores_teaching_rank', 'stats_female_male_ratio',
       'stats_number_students', 'stats_pc_intl_students',
       'stats_student_staff_ratio', 'subjects_offered', 'url'],
      dtype='object')


We don't need all those columns. The ones we need are the location, the name, the rank and the stats at the end (we will also take the nid). We notice here that we don't have access to the region but only the location. We also notice that there is no information concerning the number of international 

In [26]:
times_dataframe = times_dataframe[['location', 'name', 'nid', 'rank', 'rank_order', 'stats_number_students', 'stats_pc_intl_students', 'stats_student_staff_ratio']]

In [27]:
times_dataframe

,location,name,nid,rank,rank_order,stats_number_students,stats_pc_intl_students,stats_student_staff_ratio
0,United Kingdom,University of Oxford,468,1,10,"20,409",38%,11.2
1,United Kingdom,University of Cambridge,470,2,20,"18,389",35%,10.9
2,United States,California Institute of Technology,128779,=3,30,"2,209",27%,6.5
3,United States,Stanford University,467,=3,40,"15,845",22%,7.5
4,United States,Massachusetts Institute of Technology,471,5,50,"11,177",34%,8.7
5,United States,Harvard University,466,6,60,"20,326",26%,8.9
6,United States,Princeton University,469,7,70,"7,955",24%,8.3
7,United Kingdom,Imperial College London,472,8,80,"15,857",55%,11.4
8,United States,University of Chicago,473,9,90,"13,525",25%,6.2
9,Switzerland,ETH Zurich – Swiss Federal Institute of Techno...,479,=10,100,"19,233",38%,14.6
